In [1]:
import os
import json
import uuid
from datetime import datetime
import pandas as pd
import openai

# OpenAI API 키 설정 (환경변수로 설정되어 있어야 함)
openai.api_key = os.getenv("OPENAI_API_KEY")

def process_extracted_table_with_gpt(table_df, child_name):
    """
    OCR 결과로 생성된 table_df와 자녀 이름을 받아,
    GPT 모델을 호출하여 처방전 정보를 추출하는 함수입니다.
    출력은 아래 JSON 형식이어야 합니다.
    
    {
        "약국명": "<약국명>",
        "처방전번호": "<처방전번호>",
        "조제일자": "<조제일자>",
        "약국주소": "<약국주소>",
        "총수납금액": "<총수납금액>",
        "약품목록": [
            {
                "약품명": "<약품명>",
                "투약량": <투약량>,
                "투약횟수": <투약횟수>,
                "투약일수": <투약일수>
            },
            ...
        ]
    }
    """
    # 미리 생성한 처방전번호와 오늘 날짜 사용
    unique_id = str(uuid.uuid4())[:8]
    prescription_number = f"RX-{unique_id}"
    current_date = datetime.now().strftime('%Y-%m-%d')

    prompt = f"""
아래는 OCR로 추출된 처방전 테이블 데이터의 JSON 표현입니다:
{table_df.to_json(orient="records", force_ascii=False)}

자녀 이름: {child_name}

이 데이터를 바탕으로 다음 항목들을 추출해줘:
- 약국명: 처방전에 찍힌 약국의 이름
- 처방전번호: "{prescription_number}" (생성된 값)
- 조제일자: "{current_date}" (오늘 날짜)
- 약국주소: 처방전에 찍힌 약국의 주소
- 총수납금액: 처방전의 총수납금액 (숫자만)
- 약품목록: 각 항목은 약품명, 투약량, 투약횟수, 투약일수

출력은 반드시 아래의 JSON 형식으로 출력해줘:
{{
    "약국명": "<약국명>",
    "처방전번호": "{prescription_number}",
    "조제일자": "{current_date}",
    "약국주소": "<약국주소>",
    "총수납금액": "<총수납금액>",
    "약품목록": [
        {{
            "약품명": "<약품명>",
            "투약량": <투약량>,
            "투약횟수": <투약횟수>,
            "투약일수": <투약일수>
        }}
    ]
}}

가능한 정보가 없으면 해당 항목은 빈 문자열 또는 0으로 표시해.
    """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    gpt_response = response.choices[0].message.content.strip()
    
    try:
        result = json.loads(gpt_response)
    except Exception as e:
        raise ValueError(f"GPT 응답 JSON 파싱 오류: {e}\n응답 내용: {gpt_response}")
    
    return result

if __name__ == "__main__":
    # 테스트용 OCR 결과 예시 데이터 (실제 OCR API 결과에서 추출된 table 데이터 형식을 가정)
    data = [
        ["상호", "ABC약국"],
        ["사업장소재지", "서울시 강남구 역삼동"],
        ["합계", "총금액 10000원"],
        ["약품명", "파라세타몰"],
        # 필요한 다른 행이 있다면 추가
    ]
    table_df = pd.DataFrame(data)
    child_name = "최지아"
    
    result = process_extracted_table_with_gpt(table_df, child_name)
    print(json.dumps(result, ensure_ascii=False, indent=4))


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
